### Synthetic Bank Transaction Data Creation

In [58]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Define constants
NUM_TRANSACTIONS = 360
ACCOUNT_ID = "8010-27075685-001"

MERCHANTS_NON_RECURRENT_1 = [
    "Eateries", "Hypermarket", "Groceries", "Petrol", "Entertainment"
]

MERCHANTS_NON_RECURRENT_2 = [
    "Auto Repair", "Plumber Service", "Electrician Service"
]

DEPOSIT_SOURCE_RECURRENT_EXTRA = [
    "Freelance Income", "ATM Deposit", "Check Deposit", "Wire Transfer", "Mobile Wallet Transfer", "Cash Deposit"
]
# Randomly select only 1 DEPOSIT_SOURCE_NON_RECURRENT
DEPOSIT_SOURCE_RECURRENT_EXTRA = [random.choice(DEPOSIT_SOURCE_RECURRENT_EXTRA)]

MERCHANTS_RECURRENT_ENTERTAINMENT = ["Video Streaming", "Music Streaming", "Cloud Gaming Subscription"]
# Randomly select only 2 MERCHANTS_RECURRENT_ENTERTAINMENT
MERCHANTS_RECURRENT_ENTERTAINMENT = random.sample(MERCHANTS_RECURRENT_ENTERTAINMENT, 2)

MERCHANTS_RECURRENT_FACILITIES_1 = ["Electric Company", "Insurance", "Gym Subscription"]
MERCHANTS_RECURRENT_FACILITIES_2 = ["Water Utility", "Gas Provider", "Trash Collection Service", "Cloud Service", "Public Transport Pass"]

BANK_LOAN_RECURRENT = ["House Loan", "Vehicle Loan", "Personal Loan"]
# Randomly select 1 to 3 BANK_LOAN_RECURRENT
BANK_LOAN_RECURRENT = random.sample(BANK_LOAN_RECURRENT, random.randint(1, 3))

FAMILY_CONTRIBUTION_RECURRENT_1 = ["Spouse Allowance", "Parent Allowance"]
FAMILY_CONTRIBUTION_RECURRENT_2 = ["Children Allowance"]

# Randomly select 1 to 2 FAMILY_CONTRIBUTION_RECURRENT_1
FAMILY_CONTRIBUTION_RECURRENT_1 = random.sample(FAMILY_CONTRIBUTION_RECURRENT_1, random.randint(1, 2))
# Randomly select 0 to 1 FAMILY_CONTRIBUTION_RECURRENT_2
FAMILY_CONTRIBUTION_RECURRENT_2 = random.sample(FAMILY_CONTRIBUTION_RECURRENT_2, random.randint(1, 1))

DEPOSIT_SOURCE_RECURRENT = ["Salary Deposit"]

START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 2, 5)

# Generate random date between start and end dates
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days), seconds=random.randint(0, 86400))

# Generate recurrent transactions once per month
recurrent_transactions = {}

recurrent_transactions_list = [
        'Entertainment: random.sample(MERCHANTS_RECURRENT_ENTERTAINMENT,1)',
        'Facilities (1a): list(MERCHANTS_RECURRENT_FACILITIES_1[0])',
        'Facilities (1b): list(MERCHANTS_RECURRENT_FACILITIES_1[1])',
        'Facilities (1c): list(MERCHANTS_RECURRENT_FACILITIES_1[2])',
        'Facilities (2a): list(MERCHANTS_RECURRENT_FACILITIES_2[0])',
        'Facilities (2b): list(MERCHANTS_RECURRENT_FACILITIES_2[1])',
        'Facilities (2c): list(MERCHANTS_RECURRENT_FACILITIES_2[2])',
        'Facilities (2d): list(MERCHANTS_RECURRENT_FACILITIES_2[3])',
        'Facilities (2e): list(MERCHANTS_RECURRENT_FACILITIES_2[4])',
        'Housing Loan: list(BANK_LOAN_RECURRENT[0])',
        'Vehicle Loan: list(BANK_LOAN_RECURRENT[1])',
        'Personal Loan:list(BANK_LOAN_RECURRENT[2])',
        'Family (1): random.sample(FAMILY_CONTRIBUTION_RECURRENT_1,1)',
        'Family (2): random.sample(FAMILY_CONTRIBUTION_RECURRENT_2,1)',
        'Deposit: random.sample(DEPOSIT_SOURCE_RECURRENT,1)',
        'Deposit Extra: random.sample(DEPOSIT_SOURCE_RECURRENT_EXTRA,1)'
]

for data in recurrent_transactions_list:
    for month in range(1, 14):  # 13 months (Jan 2024 - Jan 2025)
        # Generate random hour (0-23) and minute (0-59)
        random_hour = random.randint(0, 23)
        random_minute = random.randint(0, 59)
        random_second = random.randint(0, 59)
        
        # Create datetime with both date and time
        month_date = datetime(
            2024 if month <= 12 else 2025,  # Year
            month if month <= 12 else 1,     # Month
            random.randint(1, 7),            # Day
            random_hour,                     # Hour
            random_minute,                   # Minute
            random_second                   # second
        )
        
        recurrent_transactions[month_date] = {
            data
        }

transactions = []

NON_RECURRENT_TRANSACTIONS = NUM_TRANSACTIONS-len(recurrent_transactions)

date_recurrents = list(recurrent_transactions.keys())

for date_loop in date_recurrents:
    if date_loop in recurrent_transactions:
        category=list(recurrent_transactions[date_loop])[0].replace("'","").split(":")[0]
        #merchant = list(recurrent_transactions[date_loop])[0].replace("'","").split(":")[1].strip()
        merchant=eval(list(recurrent_transactions[date_loop])[0].replace("'","").split(":")[1].strip())
        
        # Set amount based on category
        if category == "Entertainment":
            transaction_type = "Debit"
            amount = -(round(random.uniform(100, 150), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (1a)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(150, 250), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (1b)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(200, 250), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (1c)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(250, 300), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (2a)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(50, 100), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (2b)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(50, 100), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (2c)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(50, 100), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (2d)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(100, 200), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Facilities (2e)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(50, 100), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Housing Loan":
            transaction_type = "Debit"
            amount = -(round(random.uniform(1000, 1001), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Vehicle Loan":
            transaction_type = "Debit"
            amount = -(round(random.uniform(500, 501), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Personal Loan":
            transaction_type = "Debit"
            amount = -(round(random.uniform(500, 501), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Family (1)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(250, 300), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Family (2)":
            transaction_type = "Debit"
            amount = -(round(random.uniform(100, 150), 2))
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Deposit":
            transaction_type = "Credit"
            amount = round(random.uniform(5000, 5001), 2)
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
        elif category == "Deposit Extra":
            transaction_type = "Credit"
            amount = round(random.uniform(50, 150), 2)  # Keeping original range for non-recurrent
            date=date_loop
            transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])
    else:
        continue

for _ in range(NON_RECURRENT_TRANSACTIONS):
    date = random_date(START_DATE, END_DATE)
    
    transaction_type = "Debit"
    category = "Non-Recurrent Expense"
    MERCHANTS_NON_RECURRENT = ['MERCHANTS_NON_RECURRENT_1','MERCHANTS_NON_RECURRENT_2']
    if random.sample(MERCHANTS_NON_RECURRENT,1) == 'MERCHANTS_NON_RECURRENT_1':
        merchant = random.sample(MERCHANTS_NON_RECURRENT_1, random.randint(0, 1))
        try:
            merchant = merchant[0]
            amount = -(round(random.uniform(50, 60), 2))  # Keeping original range for non-recurrent
        except IndexError:
            merchant = None
            amount = 0
            continue
    else:
        merchant = random.sample(MERCHANTS_NON_RECURRENT_2, random.randint(0, 1))
        try:
            merchant = merchant[0]
            amount = -(round(random.uniform(150, 250), 2))  # Keeping original range for non-recurrent
        except IndexError:
            merchant = None
            amount = 0
            continue


    transactions.append([ACCOUNT_ID, date, transaction_type, category, merchant, amount])

# Convert to DataFrame with explicit column names
df = pd.DataFrame(transactions, columns=["accountID", "date", "transactionType", "category", "merchant", "Amount"])

# Sort transactions by date
df.sort_values(by="date", inplace=True)

# Calculate cumulative balance
df["accountBAL"] = df["Amount"].cumsum().round(2)

df["merchant"]=df["merchant"].astype(str).str.replace("[","").str.replace("]","").str.replace("'","").str.replace(",","").str.replace(",,"," ")

# Save to CSV
# df.to_csv("synthetic_current_account_transactions.csv", index=False)

print("Synthetic data generated and saved as 'synthetic_current_account_transactions.csv'.")


Synthetic data generated and saved as 'synthetic_current_account_transactions.csv'.


In [59]:
import numpy as np

df['merchant']=np.where(df['category'].str.contains('Loan'),df['category'],df['merchant'])
df['category']=np.where(df['category'].str.contains('Non-Recurrent'),df['merchant'],df['category'])
df['category']=np.where(df['category'].str.contains('Auto Repair|Plumber Service|Electrician Service'),'Maintenance/Service',df['category'])
df['category']=np.where(df['category'].str.contains('Loan'),'Loan',df['category'])
df['category']=np.where(df['category'].str.contains('Family'),'Family Allowances',df['category'])
df['category']=np.where(df['merchant'].str.contains('Video Streaming|Music Streaming|Cloud Gaming Subscription|Cloud Service'),'Bills (Entertainment)',df['category'])
df['category']=np.where(df['merchant'].str.contains('Electric Company|Water Utility|Gas Provider|Trash Collection Service'),'Bills (Utilities)',df['category'])
df['category']=np.where(df['merchant'].str.contains('Public Transport Pass|Petrol'),'Transportation',df['category'])
df['category']=np.where(df['merchant'].str.contains('Eateries|Hypermarket|Groceries'),'Food & Dining',df['category'])
df['category']=np.where(df['merchant'].str.contains('Gym Subscription|Insurance'),'Health',df['category'])


df

,accountID,date,transactionType,category,merchant,Amount,accountBAL
143,8010-27075685-001,2024-01-01 01:39:49,Debit,Loan,Personal Loan,-500.90,-500.90
268,8010-27075685-001,2024-01-01 16:05:48,Debit,Maintenance/Service,Auto Repair,-201.48,-702.38
156,8010-27075685-001,2024-01-01 23:16:18,Debit,Family Allowances,Spouse Allowance,-279.94,-982.32
257,8010-27075685-001,2024-01-02 00:38:04,Debit,Maintenance/Service,Electrician Service,-214.07,-1196.39
182,8010-27075685-001,2024-01-02 10:33:59,Credit,Deposit,Salary Deposit,5000.23,3803.84
...,...,...,...,...,...,...,...
209,8010-27075685-001,2025-01-16 02:58:25,Debit,Maintenance/Service,Auto Repair,-239.84,3083.78
242,8010-27075685-001,2025-01-17 13:40:20,Debit,Maintenance/Service,Auto Repair,-242.66,2841.12
286,8010-27075685-001,2025-01-29 08:39:10,Debit,Maintenance/Service,Electrician Service,-178.24,2662.88
262,8010-27075685-001,2025-02-01 18:54:19,Debit,Maintenance/Service,Plumber Service,-151.64,2511.24


In [60]:
df=df.rename(columns={"merchant":"description"})
df.to_csv("synthetic_current_account_transactions.csv", index=False)

In [55]:
df

NameError: name 'df' is not defined

### Create Account ID 

In [62]:
# direct query

'''
UPDATE cif
SET accountID = 
    CASE 
        -- Savings Account: Format 8010-IC-001 (starts with 8010)
        WHEN productID = 1 THEN 
            '8010-' || substr('00000000' || ic, -8) || '-' ||
            substr('000' || id, -3)
        
        -- Current Account: Format 8020-IC-001 (starts with 8020)
        WHEN productID = 2 THEN 
            '8020-' || substr('00000000' || ic, -8) || '-' ||
            substr('000' || id, -3)
    END
WHERE productID IN (1, 2)

'''


# connect to sqlite
import sqlite3
conn = sqlite3.connect('instance/flaskr.sqlite')
print("Opened database successfully")

# pandas read from sqlite
#import pandas as pd
#pd.read_sql_query("select * from 'productID(1)-transactions'", conn)

# write pandas df to sqlite table
#df = pd.read_csv('synthetic_current_account_transactions.csv')
df.to_sql('productID(1)-transactions', conn, if_exists='append', index=False)

#df=pd.read_sql_query("select * from 'productID(1)-transactions'", conn)
#df['accountBAL']=(df['transactionAMT'].cumsum()).round(2)
#df

conn.close()


Opened database successfully


In [68]:
# pandas read from sqlite
import pandas as pd
import sqlite3
conn = sqlite3.connect('instance/flaskr.sqlite')
accountID = "8010-27075685-001"

query = """

SELECT *, ROUND((total_amount/denominator)*100, 2) as pct FROM
(SELECT *, SUM(ABS(total_amount)) OVER() as denominator FROM 
(SELECT category, COUNT(*) as transaction_count, SUM(ABS(Amount)) as total_amount, ROUND(AVG(ABS(Amount)), 2) as average_amount
FROM 
(SELECT * FROM "productID(1)-transactions" WHERE accountID = '"""+ accountID + """' AND transactionTYPE = "Debit" AND date >= date('now', '-30 days') ORDER BY ROWID DESC)
GROUP BY category))

"""
pd.read_sql_query(query, conn)

,category,transaction_count,total_amount,average_amount,denominator,pct
0,Bills (Entertainment),2,232.55,116.28,5280.29,4.40
1,Bills (Utilities),4,453.00,113.25,5280.29,8.58
2,Family Allowances,2,397.43,198.71,5280.29,7.53
3,Health,2,499.35,249.68,5280.29,9.46
4,Loan,3,2001.56,667.19,5280.29,37.91
5,Maintenance/Service,8,1629.20,203.65,5280.29,30.85
6,Transportation,1,67.20,67.20,5280.29,1.27


In [47]:
query = """



SELECT *, SUM(ABS(Amount)) as denominator FROM
(SELECT * FROM "productID(1)-transactions" WHERE accountID = '"""+ accountID + """' AND transactionTYPE = "Debit" AND date >= date('now', '-30 days') ORDER BY ROWID DESC)


"""
pd.read_sql_query(query, conn)

,accountID,date,transactionTYPE,description,Amount,accountBAL,category,status,denominator
0,8010-27075685-001,2025-02-04 19:57:05,Debit,Plumber Service,-170.15,2341.09,Maintenance/Service,Completed,5280.29
